### Use case for NFDInspector
In this use case, we will explore the capabilities of NFDInspector a little further. We will go through a typical application. In our scenario, a collection of stereo glass plates was digitized as part of a digitization project. The associated metadata was partly generated from existing legacy data and partly created by new indexing.
NDFInspector for LIDO-xml offers the possibility to adapt the parameters of the analysis to the respective use case. For this purpose, we will take a closer look at the configuration file and the setting options within it.<br>
The LIDO xml files were created and collected in the /LIDO_xml folder. There are a total of 2,063 LIDO-records in separate xml files. The existing data was generated from the portal __[westfalen.museum-digital.de](https://westfalen.museum-digital.de/objects?s=collection:670)__. For more information on the specifications of the LIDO format, please refer to the __[documentation](https://cidoc.mini.icom.museum/working-groups/lido/lido-overview/about-lido/what-is-lido/)__.

### Initialization
To begin the inspection, the NFDInspector package is imported and the lido_inspector is initialized. English is selected as the output language.

In [3]:
from nfdinspector.lido_inspector import LIDOInspector

lido_inspector = LIDOInspector(error_lang='en')

### Configuration
At this point, we will take a closer look at the configuration and therefore the configuration file. As there is no ready-made file available, the standard configuration is read from the ``LIDOInspector``. The dictionary created in this way is saved as a JSON file and can also be displayed in the notebook.

In [ ]:
import json

lido_config = lido_inspector.configuration
with open('lido_config.json', "w") as outfile:
    json.dump(lido_config, outfile, indent=4)

lido_config